In [1]:
!pwd
!ls /kaggle/input/million-song-dataset-spotify-lastfm/'Music Info.csv'
!cp /kaggle/input/million-song-dataset-spotify-lastfm/'Music Info.csv' /kaggle/input/million-song-dataset-spotify-lastfm/music_info.csv

/kaggle/working
'/kaggle/input/million-song-dataset-spotify-lastfm/Music Info.csv'
cp: cannot create regular file '/kaggle/input/million-song-dataset-spotify-lastfm/music_info.csv': Read-only file system


In [2]:
!pip install -q audiomentations tables pydub spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 10.7 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import requests
import io
from IPython.display import Audio, display
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import re
import json
import pickle as pkl
import base64
import time
import urllib.parse
from tqdm import tqdm
import sqlite3
import spotipy
from sklearn.model_selection import train_test_split
from spotipy.oauth2 import SpotifyClientCredentials
from requests.exceptions import ReadTimeout
from tqdm import tqdm
import sys
from tenacity import retry, wait_exponential, stop_after_attempt, RetryError, retry_if_exception_type
import soundfile as sf
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, AddGaussianSNR, ClippingDistortion, Gain
from pydub import AudioSegment
import random

In [4]:
# Get dataframe from kaggle dataset
music_info_df = pd.read_csv('/kaggle/input/million-song-dataset-spotify-lastfm/Music Info.csv')
music_info_df.head(10)
print(len(music_info_df))

50683


In [5]:
print(music_info_df.iloc[0,:])

track_id                                              TRIOREW128F424EAF0
name                                                      Mr. Brightside
artist                                                       The Killers
spotify_preview_url    https://p.scdn.co/mp3-preview/4d26180e6961fd46...
spotify_id                                        09ZQ5TmUG8TSL56n0knqrj
tags                   rock, alternative, indie, alternative_rock, in...
genre                                                                NaN
year                                                                2004
duration_ms                                                       222200
danceability                                                       0.355
energy                                                             0.918
key                                                                    1
loudness                                                           -4.36
mode                                               

In [6]:
# Function to get spotify audio from row of DF
def get_raw_audio(row, which=3):
    try:
        url = row[which]
        response = requests.get(url)
        audio_data = io.BytesIO(response.content)
        return audio_data
    except requests.RequestException as e:
        print(f"Error downloading {url}: {e}")

# Play the audio in jupyter
def play_audio(row):
    display(Audio(get_raw_audio(row).read(), autoplay=True))

def get_audio_data(row, which=3):
    y, sr = librosa.load(get_raw_audio(row, which))
    return y, sr

# Function to get the mel spectrogram for each song in each row of a dataframe
def get_spectrogram(row, which):
    audio_data = get_raw_audio(row, which)
    y, sr = librosa.load(audio_data, sr=None)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    S_DB = librosa.power_to_db(S, ref=np.max)
    return pkl.dumps((S_DB, sr))        

# Function to get chromagram. Return serialized data and sampling rate
def get_chromagram(row, which):
    audio_data = get_raw_audio(row, which)
    y, sr = librosa.load(audio_data)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    return pkl.dumps((chroma, sr))

In [7]:
# play_audio(music_info_df.iloc[0,:])

In [8]:
# len(music_info_df)

In [9]:
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from pydub import AudioSegment

# Define exceptions to handle with retry
class RateLimitError(Exception):
    def __init__(self, message, retry_after=None):
        super().__init__(message)
        self.retry_after = retry_after

# Function to clip audio to 10s (10000 ms by default)
def clip_audio(audio_data, segment_length = 10000, sr=22050):
    audio = AudioSegment.from_file(audio_data)

    if len(audio) < segment_length:
        raise ValueError("Audio is shorter than the segment length.")

    # Randomly generate start point at least 10s from the end of the song.
    start = np.random.randint(0, len(audio) - segment_length)
    audio_segment = audio[start:start + segment_length]

    # Return clip in format importable to Librosa
    audio_segment_io = io.BytesIO()
    audio_segment.export(audio_segment_io, format="wav")
    audio_segment_io.seek(0)
    return audio_segment_io # Export (with pointer at start of audio)

# Function to download and process a single audio file
@retry(stop=stop_after_attempt(10), wait=wait_exponential(multiplier=1, min=4, max=60), retry=retry_if_exception_type(RateLimitError))
def download_and_process_audio(url, sr=22050):
    try:
        response = requests.get(url)
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")
        return None   
     
    if response.status_code == 429:
        retry_after = response.headers.get('Retry-After', 'Not provided')
        print(f"Rate limit exceeded. Retry-After: {retry_after} seconds")
        raise RateLimitError("Rate limit exceeded", retry_after)
        
    response.raise_for_status()  # Check if the request was successful
    # get audio data from url
    audio_data = io.BytesIO(response.content)
    # feed into audio clipping function
    audio_data_clip = clip_audio(audio_data)


    y, sr = librosa.load(audio_data_clip, sr=sr)
    #mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
    #mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    #time.sleep(0.5)
    return y, sr
    #return mel_spectrogram_db


# Function to apply download_and_process_audio to each row
def process_row(row, sr=22050):
    url = row['spotify_preview_url']
    if url:
        return download_and_process_audio(url, sr)
    return None

# Function to download and process all audio files in the DataFrame
def download_and_process_all_audio(df, url_column, output_dir, batch_size=00, start_batch=0, sr=22050):
    os.makedirs(output_dir, exist_ok=True)
    # Dataframe to add audio too
    df['processed_audio'] = None

    num_batches = (len(df) // batch_size)
    for batch_num in range(start_batch, num_batches):
        batch_file_path = os.path.join(output_dir, f"batch_{batch_num + 1}.pkl")

        # Check if the batch has already been processed
        if os.path.exists(batch_file_path):
            print(f"Batch {batch_num + 1} already processed. Skipping...")
            continue

        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(df))
        batch_df = df.iloc[start_idx:end_idx]

        # Pass each row to other functions with progress bar. Output to pickle
        tqdm.pandas(desc=f"Processing batch {batch_num + 1}/{num_batches}")
        batch_df['processed_audio'] = batch_df.progress_apply(lambda row: process_row(row, sr), axis=1).copy()
        batch_df.to_pickle(batch_file_path)

        # Combine progressively each batch with overall df. Pickle.
        #df.iloc[start_idx:end_idx] = batch_df
        #df.to_pickle(os.path.join(output_dir, "full_dataframe.pkl"))
        time.sleep(10)


def combine_batches(pickle_dir, output_file=None):
    dataframes = []

    # Go through all pickle files taht start with batch
    for filename in sorted(os.listdir(pickle_dir)):
        if filename.startswith('batch_') and filename.endswith('.pkl'):
            file_path = os.path.join(pickle_dir, filename)
            # Load the DataFrame and append to the list
            df = pd.read_pickle(file_path)
            dataframes.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Save the combined DataFrame to a new pickle file (optional)
    if output_file:
        combined_df.to_pickle(output_file)

    return combined_df

In [10]:
def process_and_store_waveforms(df, url_column='spotify_preview_url', sr=22050):
    waveforms = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing audio"):
        url = row[url_column]
        if pd.notnull(url):
            try:
                response = requests.get(url)
                if response.status_code == 429:
                    print(f"Rate limit at row {idx}. Skipping...")
                    waveforms.append(None)
                    continue

                response.raise_for_status()
                audio_data = io.BytesIO(response.content)
                audio_data_clip = clip_audio(audio_data)
                y, _ = librosa.load(audio_data_clip, sr=sr)
                waveforms.append(y)
            except Exception as e:
                print(f"Error at row {idx}: {e}")
                waveforms.append(None)
        else:
            waveforms.append(None)

    df['waveform'] = waveforms
    return df

In [11]:
import gc
gc.collect()

0

In [ ]:
# df = music_info_df.iloc[10000:20000, :].copy()
df = music_info_df.iloc[0:5, :].copy()

# Specify the output directory for downloaded files
output_dir = "/kaggle/working/download/"
os.makedirs(output_dir, exist_ok=True)
df = process_and_store_waveforms(df)

# Download and process all audio files and update the DataFrame with the processed data
# download_and_process_all_audio(df, 'spotify_preview_url', output_dir, batch_size=1000)

Processing audio: 100%|██████████| 5/5 [00:17<00:00,  3.58s/it]


In [13]:
#df = combine_batches('/Users/reggiebain/erdos/song-similarity-erdos-old/data/audio_download', 
#                     '/Users/reggiebain/erdos/song-similarity-erdos-old/data/audio_download/all_batches_df.pkl')

In [14]:
# print(df['waveform'].head(5))
# print(len(df['waveform'][0]))
# print(len(df['waveform'][2]))
# print(len(df['waveform'][789]))

In [15]:
df.head()
# df.to_csv('/kaggle/working/my_saved_data.csv', index=False)


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,waveform
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4,"[-0.283236, -0.55442226, -0.5227047, -0.644138..."
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4,"[0.31897366, 0.33190382, 0.20763808, 0.2647782..."
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4,"[0.013973154, 0.053070467, 0.16098025, 0.25506..."
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4,"[0.17191999, 0.25843543, 0.15508036, 0.1254905..."
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4,"[0.017730419, 0.1649383, 0.12062906, 0.0725574..."


In [16]:
df.to_pickle("/kaggle/working/download/audio30k-34k.pkl")

In [17]:
# df1 = pd.read_pickle("/kaggle/working/download/audio.pkl")

In [18]:
# df1.head()

In [19]:
df.columns

Index(['track_id', 'name', 'artist', 'spotify_preview_url', 'spotify_id',
       'tags', 'genre', 'year', 'duration_ms', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'waveform'],
      dtype='object')